[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/agent-memory.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239417-lesson-7-agent-with-memory)

# Agent memory

## Review

Previously, we built an agent that can:

* `act` - let the model call specific tools
* `observe` - pass the tool output back to the model
* `reason` - let the model reason about the tool output to decide what to do next (e.g., call another tool or just respond directly)

![Screenshot 2024-08-21 at 12.45.32 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab7453080e6802cd1703_agent-memory1.png)

## Goals

Now, we're going extend our agent by introducing memory.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


We'll use [LangSmith](https://docs.smith.langchain.com/) for [tracing](https://docs.smith.langchain.com/concepts/tracing).

In [3]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

LANGCHAIN_API_KEY: ··········


This follows what we did previously.

In [4]:
from langchain_openai import ChatOpenAI

# Define the tools
def check_ip(ip_address: str) -> str:
    """Check if an IP address is malicious."""
    malicious_ips = ["203.0.113.42", "198.51.100.17"]
    return "malicious" if ip_address in malicious_ips else "clean"

def check_domain(domain: str) -> str:
    """Check if a domain is malicious."""
    malicious_domains = ["malicious.example.com"]
    return "malicious" if domain in malicious_domains else "clean"

def suggest_remediation(threat: str) -> str:
    """Suggest remediation steps based on the threat."""
    remediation_steps = {
        "malicious_ip": "Block the IP address and monitor network traffic.",
        "malicious_domain": "Block the domain at the DNS level and monitor for phishing attempts.",
        "data_exfiltration": "Isolate affected systems and conduct forensic analysis."
    }
    return remediation_steps.get(threat, "No specific remediation steps found.")


# List of tools
tools = [check_ip, check_domain, suggest_remediation]

# Set up the chat model and bind the tools
llm = ChatOpenAI(model="gpt-4")
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


In [5]:
# Define the assistant function
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message guiding the assistant's behavior
sys_msg = SystemMessage(content=(
    "You are a cybersecurity assistant specialized in Data Loss Prevention (DLP). "
    "You can summarize incidents, check indicators of compromise (IOCs), and suggest remediation steps. "
    "Use available tools when necessary."
))

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


In [8]:
# Build the graph
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")
dlp_agent_graph = builder.compile()


## Memory

Let's run our agent, as before.

In [18]:
messages = [HumanMessage(content="someone sent a pdf from their personal computer")]
messages = dlp_agent_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

someone sent a pdf from their personal computer
================================== Ai Message ==================================

In order to analyze this incident thoroughly, I need more specifics. Could you please provide additional information such as:

1. The destination of the sent PDF. Was it sent to an external or internal entity?
2. The content of the PDF. Did it contain sensitive or company-related data?
3. The method of transmission. Was it via a company email, personal email or some other method?
4. The IP address or domain involved in the incident.

Once I have this information, I can then proceed with the investigation, by checking the IP or domain for malicious activity and suggesting remediation steps if necessary.


Now, let's multiply by 2!

In [19]:
messages = [HumanMessage(content="which can i use for my dlp case?")]
messages = dlp_agent_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

which can i use for my dlp case?
================================== Ai Message ==================================

Based on your role as a cybersecurity assistant specialized in Data Loss Prevention (DLP), you can use the following tools:

1. **Check IP**: This function allows you to check if an IP address is malicious. This is useful in incident investigation when you have an IP address that you suspect may be associated with malicious activities. This tool can provide important information about the reputation of the IP address.

2. **Check Domain**: This function lets you check if a domain is malicious. Similar to the Check IP tool, this is valuable in investigations where a particular domain is suspected of being linked to malicious activities.

3. **Suggest Remediation**: This function suggests remediation steps based on the threat. After identifying a threat, this tool can provide guidance on how to

We don't retain memory of 7 from our initial chat!

This is because [state is transient](https://github.com/langchain-ai/langgraph/discussions/352#discussioncomment-9291220) to a single graph execution.

Of course, this limits our ability to have multi-turn conversations with interruptions.

We can use [persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) to address this!

LangGraph can use a checkpointer to automatically save the graph state after each step.

This built-in persistence layer gives us memory, allowing LangGraph to pick up from the last state update.

One of the easiest checkpointers to use is the `MemorySaver`, an in-memory key-value store for Graph state.

All we need to do is simply compile the graph with a checkpointer, and our graph has memory!

In [14]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

When we use memory, we need to specify a `thread_id`.

This `thread_id` will store our collection of graph states.

Here is a cartoon:

* The checkpointer write the state at every step of the graph
* These checkpoints are saved in a thread
* We can access that thread in the future using the `thread_id`

![state.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e0e9f526b41a4ed9e2d28b_agent-memory2.png)


In [16]:
# Specify a thread
config = {"configurable": {"thread_id": "1"}}

# Specify an input
messages = [HumanMessage(content="What is xsoar and symantec?")]

# Run
messages = react_graph_memory.invoke({"messages": messages},config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

I'm sorry for any confusion, but as a cybersecurity assistant, I specialize in data loss prevention, checking indicators of compromise, and suggesting remediation steps. I'm not equipped to perform mathematical operations. Can I assist you with something related to cybersecurity?
================================ Human Message =================================

What is xsoar and symantec?
================================== Ai Message ==================================

XSOAR and Symantec are both key players in the field of cybersecurity, but they serve different functions:

1. XSOAR (Cortex XSOAR): This is a comprehensive Security Orchestration, Automation and Response (SOAR) platform that unifies case management, automation, real-time collaboration and threat intel management. It helps security teams manage and

If we pass the same `thread_id`, then we can proceed from from the previously logged state checkpoint!

In this case, the above conversation is captured in the thread.

The `HumanMessage` we pass (`"Multiply that by 2."`) is appended to the above conversation.

So, the model now know that `that` refers to the `The sum of 3 and 4 is 7.`.

In [17]:
messages = [HumanMessage(content="which can i use for my dlp case?")]
messages = react_graph_memory.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

I'm sorry for any confusion, but as a cybersecurity assistant, I specialize in data loss prevention, checking indicators of compromise, and suggesting remediation steps. I'm not equipped to perform mathematical operations. Can I assist you with something related to cybersecurity?
================================ Human Message =================================

What is xsoar and symantec?
================================== Ai Message ==================================

XSOAR and Symantec are both key players in the field of cybersecurity, but they serve different functions:

1. XSOAR (Cortex XSOAR): This is a comprehensive Security Orchestration, Automation and Response (SOAR) platform that unifies case management, automation, real-time collaboration and threat intel management. It helps security teams manage and

## LangGraph Studio

--

**⚠️ DISCLAIMER**

*Running Studio currently requires a Mac. If you are not using a Mac, then skip this step.*

--

Load the `agent` in the UI, which uses `module-1/studio/agent.py` set in `module-1/studio/langgraph.json`.